In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Build Zeitghost Custom Container Image for MLOps 
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

<center>
<img src="imgs/zghost_overview_pipeline_steps.png" width="1200"/>
</center>

Vertex Pipelines are effectively running containerized executions for each step (component) in the pipeline workflow. To simplify and reduce container creation overhead, we've included this notebook so that you can create a single custom container image that can be used for the pipeline in notebook 05-gdelt-pipelines. 

Whilst you don't necessarily have to use a custom container, the alternative is to use a base container image and pass a list of packages required to run each component step in the pipeline definition.

The previous notebooks:

1. [Setup Vertex Vector Store](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
2. [GDELT DataOps](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
3. [Vector Store Index Loader](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)
4. [Alternative document format embeddings](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb)

Have shown how you can perform a one-off extraction, index, and load into the Matching Engine Vector Store and how you can use a Langchain Agent to query the Vector Store using natural language queries. 

However, given the nature of GDELT data it may become critical to orchestrate and schedule a regularly occuring update of the Index with new vectors, as new GDELT entity and event data becomes available for a given topic or actor. 

To address this challenge, we've created a Vertex AI pipeline which will modularize, orchestrate, and only update the existing Matching Engine Index with new vectors when new data is extracted - the [GDELT Pipelines](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb) notebook shows how to create this pipeline. 

In order to create this pipeline, you'll need to containerize the environment dependencies for all of these differnet components to run - to streamline this process, you can use this notebook to generate the container to use with the pipeline. 

If your environment changes and you're extending your application, we also show how you can create a [Cloud Build Trigger](https://cloud.google.com/build/docs/automating-builds/create-manage-triggers) to automatically rebuild this container image each time new code changes are pushed to your `main` branch.

---

### Objectives

In this notebook, the steps performed include:

- Create a `Dockerfile_gdelt` image to use with Vertex AI Pipelines components
- Choose to build the custom container image either:
    - Locally with `Docker`
    - Using Cloud services with [Cloud Build](https://cloud.google.com/build/docs)
- Cloud Build [trigger](https://cloud.google.com/build/docs/automating-builds/create-manage-triggers) setup to rebuild image each time pushed to `main` branch and tag with `latest`
- The built image will be used as the base image in pipeline steps

### Costs
This tutorial uses billable components of Google Cloud:

* Cloud Build
* Artifact Registry

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started
**Colab only:** Uncomment the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Make sure you edit the values below
Each time you run the notebook for the first time with new variables, you just need to edit the actor prefix and version variables below. They are needed to grab all the other variables in the notebook configuration.

In [3]:
# CREATE_NEW_ASSETS        = True # True | False
ACTOR_PREFIX             = "ggl"
VERSION                  = 'v1'

# print(f"CREATE_NEW_ASSETS  : {CREATE_NEW_ASSETS}")
print(f"ACTOR_PREFIX       : {ACTOR_PREFIX}")
print(f"VERSION            : {VERSION}")

ACTOR_PREFIX       : ggl
VERSION            : v1


### Load configuration settings from setup notebook
> Set the constants used in this notebook and load the config settings from the `00-env-setup.ipynb` notebook.

In [4]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

BUCKET_NAME              = f'zghost-{ACTOR_PREFIX}-{VERSION}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)

print(f"BUCKET_NAME        : {BUCKET_NAME}")
print(f"BUCKET_URI         : {BUCKET_URI}")

BUCKET_NAME        : zghost-ggl-v1
BUCKET_URI         : gs://zghost-ggl-v1

PROJECT_ID               = "cpg-cdp"
PROJECT_NUM              = "939655404703"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "genai-haystack-vpc"

CREATE_NEW_ASSETS        = "True"
ACTOR_PREFIX             = "ggl"
VERSION                  = "v1"
ACTOR_NAME               = "google"
ACTOR_CATEGORY           = "cyber security"

BUCKET_NAME              = "zghost-ggl-v1"
EMBEDDING_DIR_BUCKET     = "zghost-ggl-v1-emd-dir"

BUCKET_URI               = "gs://zghost-ggl-v1"
EMBEDDING_DIR_BUCKET_URI = "gs://zghost-ggl-v1-emd-dir"

VPC_NETWORK_FULL         = "projects/939655404703/global/networks/genai-haystack-vpc"

ME_INDEX_NAME            = "vectorstore_ggl_v1"
ME_INDEX_ENDPOINT_NAME   = "vectorstore_ggl_v1_endpoint"
ME_DIMENSIONS            = "768"

MY_BQ_DATASET            = "zghost_ggl_v1"
MY_BQ_TRENDS_DATASET     = "zgho

### Container Image Variables

In [6]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/Trendspotting-5.0'

In [7]:
DOCKERNAME                = f'Dockerfile_gdelt'

REPOSITORY                = f'zghost-{ACTOR_PREFIX}'
IMAGE_NAME                = f'gdelt-pipe-{VERSION}'

REMOTE_IMAGE_NAME         = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME}"

print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

DOCKERNAME        = Dockerfile_gdelt
REPOSITORY        = zghost-ggl
IMAGE_NAME        = gdelt-pipe-v1
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/cpg-cdp/zghost-ggl/gdelt-pipe-v1


### Create Artifact Repository
If you don't have an existing artifact repository, create one using the gcloud command below

In [8]:
! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

Create request issued for: [zghost-ggl]
Waiting for operation [projects/cpg-cdp/locations/us-central1/operations/d9e2cd
45-6e91-41d7-a3e8-0bfeb4eaa942] to complete...done.                            
Created repository [zghost-ggl].


To take a quick anonymous survey, run:
  $ gcloud survey



## Local Docker build
Provide a name for your dockerfile and make sure you are authenticated

In [9]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


Create your Dockerfile 

In [10]:
%%writefile {DOCKERNAME}

FROM python:3.10

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

ADD zeitghost $APP_HOME/zeitghost

RUN ls zeitghost

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

Overwriting Dockerfile_gdelt


### Build Image Locally

In [11]:
!docker build -t $REMOTE_IMAGE_NAME -f $DOCKERNAME .

Sending build context to Docker daemon  917.9MB
Step 1/10 : FROM python:3.10
 ---> df5a406e137e
Step 2/10 : ENV PYTHONUNBUFFERED True
 ---> Using cache
 ---> 144498c33d3f
Step 3/10 : ENV APP_HOME /workspace
 ---> Using cache
 ---> 6ab22fd04756
Step 4/10 : WORKDIR $APP_HOME
 ---> Using cache
 ---> 072b99581881
Step 5/10 : COPY requirements.txt $APP_HOME/requirements.txt
 ---> fb9e5ea9261d
Step 6/10 : RUN pip install --upgrade pip
 ---> Running in 1f2e3e21c625
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Removing intermediate container 1f2e3e21c625
 ---> de2fb28304ac
Step 7/10 : RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt
 ---> Running in 5e70875e93cd
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 23.4 MB/

Once your container has finished building, now push it to the GCP Artifact Registry - once it's pushed, it can be used in the pipelines

In [12]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/cpg-cdp/zghost-ggl/gdelt-pipe-v1]

f8b32d74: Preparing 
9a114477: Preparing 
4acce058: Preparing 
36b0ef11: Preparing 
a5623b13: Preparing 
ebcc8d1b: Preparing 
6ff19cf9: Preparing 
23a5d198: Preparing 
66bcf3b5: Preparing 
420135e3: Preparing 
f5c5ff9f: Preparing 
24a95c80: Preparing 
a114477: Pushed     913MB/879.8MBost-thd/gdelt-pipe-v3 Alatest: digest: sha256:00239e7a62edeb992885cd5e9916fcf93d6b999908a5f3eed8ccf2dca3773403 size: 3056
